# 📈 PyKwant Tutorial: Yield Curves and Rates
This notebook explores `pykwant.rates`, the module responsible for interest rate modelling, discounting, and yield curve construction.

In the spirit of functional programming, `pykwant` treats a Yield Curve as a simple function (a **Closure**) that maps a date to a Discount Factor. This creates a lightweight and composable API.

## 1. Setup and Imports

In [1]:
from datetime import date
from pykwant import rates, dates

# Helper for display
def print_pct(label: str, value: float):
    print(f"{label}: {value:.2%}")

## 2. Constructing a Yield Curve
We construct a curve using `create_curve_from_discount_factor`. This represents a **Bootstrapped Curve** where we already know the discount factors at specific pillar dates.

**Scenario**: We have market data for Spot, 1-Year, and 2-Years.

In [2]:
ref_date = date(2025, 1, 1)

# Market Data Pillars
dates_list = [
    date(2025, 1, 1),  # Spot
    date(2026, 1, 1),  # 1 Year
    date(2027, 1, 1)   # 2 Years
]

# Discount Factors (DF) corresponding to the dates
# DF = 1 / (1 + r)^t
dfs_list = [
    1.0000,  # Spot DF is always 1
    0.9523,  # Implies ~5% rate
    0.9000   # Implies lower long-term rate
]

# Create the curve function
# By default, this uses Log-Linear Interpolation on Discount Factors
yield_curve = rates.create_curve_from_discount_factor(
    reference_date=ref_date,
    dates_list=dates_list,
    dfs_list=dfs_list,
    day_count=dates.act_365
)

# The result 'yield_curve' is a function: Date -> float
print(f"Curve Type: {type(yield_curve)}")

Curve Type: <class 'function'>


## 3. Discounting and Present Value
The primary use of a yield curve is to calculate the **Present Value (PV)** of future cash flows.

### Getting Discount Factors
We can call the curve function directly to get DFs for any date (interpolated automatically).

In [3]:
# 1. Exact Pillar Date
df_1y = yield_curve(date(2026, 1, 1))
print(f"DF at 1Y (Pillar):     {df_1y:.4f}")

# 2. Interpolated Date (18 months)
target_date = date(2026, 7, 1)
df_18m = yield_curve(target_date)
print(f"DF at 1.5Y (Interp.):  {df_18m:.4f}")

DF at 1Y (Pillar):     0.9523
DF at 1.5Y (Interp.):  0.9260


### Calculating PV
The `present_value` function is a helper that performs the multiplication $PV = Amount \times DF(t)$.

In [4]:
future_cash = 1000.0
payment_date = date(2027, 1, 1) # 2 Years out

pv = rates.present_value(future_cash, payment_date, yield_curve)

print(f"\nFuture Value: ${future_cash:,.2f}")
print(f"Present Value: ${pv:,.2f}")


Future Value: $1,000.00
Present Value: $900.00


## 4. Extracting Rates
While Discount Factors are used for pricing, humans prefer thinking in terms of **Rates**. `pykwant` provides pure functions to extract these from the curve.
### Zero Rates (Spot Rates)
The zero rate is the continuously compounded rate $r$ such that $DF = e^{-r \tau}$.
`rates.zero_rates` calculates this for a specific date.

In [5]:
# Rate for 1 Year
r_1y = rates.zero_rates(yield_curve, ref_date, date(2026, 1, 1))
print_pct("Zero Rate 1Y", r_1y)

# Rate for 2 Years
r_2y = rates.zero_rates(yield_curve, ref_date, date(2027, 1, 1))
print_pct("Zero Rate 2Y", r_2y)

Zero Rate 1Y: 4.89%
Zero Rate 2Y: 5.27%


### Forward Rates
The forward rate is the implied rate applicable between two future dates ($T_1$ and $T_2$).
`rates.forward_rate` computes this via the ratio of discount factors: $Fwd \approx (DF_{start} / DF_{end} - 1) / \tau$.

In [6]:
# Forward rate between Year 1 and Year 2
d1 = date(2026, 1, 1)
d2 = date(2027, 1, 1)

fwd_1y_1y = rates.forward_rate(yield_curve, start_date=d1, end_date=d2)

print("\n--- Forward Rates ---")
print(f"Interval: {d1} to {d2}")
print_pct("Forward Rate", fwd_1y_1y)


--- Forward Rates ---
Interval: 2026-01-01 to 2027-01-01
Forward Rate: 5.81%


## 5. Compounding Factors
Sometimes we need to project a current amount into the future using a known rate. The `compound_factor` function handles both continuous and discrete compounding.

In [7]:
rate = 0.05 # 5%
time = 2.0  # 2 Years

# Continuous: e^(rt)
comp_cont = rates.compound_factor(rate, time, frequency=0)

# Discrete (Annual): (1 + r)^t
comp_annual = rates.compound_factor(rate, time, frequency=1)

# Discrete (Semiannual): (1 + r/2)^(2t)
comp_semi = rates.compound_factor(rate, time, frequency=2)

print("\n--- Compounding 5% over 2 Years ---")
print(f"Continuous:  {comp_cont:.6f}")
print(f"Annual:      {comp_annual:.6f}")
print(f"Semiannual:  {comp_semi:.6f}")


--- Compounding 5% over 2 Years ---
Continuous:  1.105171
Annual:      1.102500
Semiannual:  1.103813
